In [68]:
#NEEDED LIBRARY
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa
import librosa.display
import librosa
%matplotlib inline
import pandas as pd

In [69]:
metadata=pd.read_csv("C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_training_part1/labels.csv")#READ TRAINING PART CSV FOR KNOWN ALGORITHM

In [70]:
metadata_un=pd.read_csv("C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_unseen/labels.csv")#READ TRAINING PART CSV FOR UNKNOWN ALGORITHM



In [71]:
#MFCC FEATURE EXTRCTION
def feature_extractor(file):
    data,sample_rate=librosa.load(file,res_type='kaiser_fast')
    mfccs=librosa.feature.mfcc(y=data,sr=sample_rate,n_mfcc=60)    
    mfccs_scaled_features=np.mean(mfccs.T,axis=0)
    return mfccs_scaled_features

In [72]:
import os
audio_dataset_path="C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_training_part1"#SET PATH FOR TRAINING PART FOR KNOWN ALGORITHM
audio_dataset_path_un="C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_unseen"#SET PATH FOR TRAINING PART FOR UNKNOWN ALGORITHM

In [22]:
#FEATURE EXTRCTION FOR KNOWN ALGORITHM
import numpy as np
from tqdm import tqdm
extracted_features=[]
for index,row in tqdm(metadata.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_dataset_path),str(row["track"]))
    final_class_label=row["algorithm"]
    data=feature_extractor(file_name)
    extracted_features.append([data,final_class_label])


5000it [06:45, 12.32it/s]


In [23]:
#FEATURE EXTRCTION FOR KNOWN ALGORITHM
for index,row in tqdm(metadata_un.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_dataset_path_un),str(row["track"]))
    final_class_label=row["algorithm"]
    data=feature_extractor(file_name)
    extracted_features.append([data,final_class_label])

1000it [01:32, 10.79it/s]


In [103]:
#SHUFFLE DATASET RANDOMLY
import random
random.shuffle(extracted_features)

In [104]:
extracted_features_df=pd.DataFrame(extracted_features,columns=['track','algorithm']) 

In [105]:
x=np.array(extracted_features_df['track'].tolist())
y=np.array(extracted_features_df['algorithm'].tolist())

In [106]:
y=np.array(pd.get_dummies(y))

In [107]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.001,random_state=0)

In [89]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics

In [90]:
num_labels=y.shape[1]

In [91]:
model=Sequential()
###first layer
model.add(Dense(100,input_shape=(60,)))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###second layer
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dropout(0.5))
###third layer
model.add(Dense(100))
model.add(Activation('relu'))
model.add(Dropout(0.5))

###final layer
model.add(Dense(num_labels))
model.add(Activation('softmax'))

In [92]:
model.compile(loss='categorical_crossentropy',metrics=['accuracy'],optimizer='adam')

In [93]:
## Trianing my model
from tensorflow.keras.callbacks import ModelCheckpoint
from datetime import datetime 

num_epochs = 100
num_batch_size = 32

checkpointer = ModelCheckpoint(filepath='C:/Users/admin/Desktop/saved_models/audio_classification_part1.hdf5', 
                               verbose=1, save_best_only=True)
start = datetime.now()

model.fit(x_train, y_train, batch_size=num_batch_size, epochs=num_epochs, validation_data=(x_test, y_test), callbacks=[checkpointer], verbose=1)


duration = datetime.now() - start
print("Training completed in time: ", duration)

Epoch 1/100
180/188 [===========================>..] - ETA: 0s - loss: 9.4530 - accuracy: 0.2311 
Epoch 1: val_loss improved from inf to 1.67327, saving model to C:/Users/admin/Desktop/saved_models\audio_classification_part1.hdf5
188/188 [==============================] - 1s 5ms/step - loss: 9.1671 - accuracy: 0.2327 - val_loss: 1.6733 - val_accuracy: 0.5000
Epoch 2/100
184/188 [============================>.] - ETA: 0s - loss: 1.5768 - accuracy: 0.3804
Epoch 2: val_loss improved from 1.67327 to 0.93874, saving model to C:/Users/admin/Desktop/saved_models\audio_classification_part1.hdf5
188/188 [==============================] - 1s 4ms/step - loss: 1.5707 - accuracy: 0.3822 - val_loss: 0.9387 - val_accuracy: 0.8333
Epoch 3/100
186/188 [============================>.] - ETA: 0s - loss: 1.0355 - accuracy: 0.5628
Epoch 3: val_loss improved from 0.93874 to 0.44703, saving model to C:/Users/admin/Desktop/saved_models\audio_classification_part1.hdf5
188/188 [==============================] -

Epoch 27/100
180/188 [===========================>..] - ETA: 0s - loss: 0.1981 - accuracy: 0.9299
Epoch 27: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 4ms/step - loss: 0.1977 - accuracy: 0.9299 - val_loss: 6.3825e-05 - val_accuracy: 1.0000
Epoch 28/100
179/188 [===========================>..] - ETA: 0s - loss: 0.2030 - accuracy: 0.9248
Epoch 28: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 4ms/step - loss: 0.2013 - accuracy: 0.9266 - val_loss: 6.2387e-04 - val_accuracy: 1.0000
Epoch 29/100
177/188 [===========================>..] - ETA: 0s - loss: 0.1861 - accuracy: 0.9299
Epoch 29: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 4ms/step - loss: 0.1866 - accuracy: 0.9293 - val_loss: 0.0011 - val_accuracy: 1.0000
Epoch 30/100
184/188 [============================>.] - ETA: 0s - loss: 0.1870 - accuracy: 0.9299
Epoch 30: val_loss did not improve from 0.00000
188/188 [======

172/188 [==========================>...] - ETA: 0s - loss: 0.1752 - accuracy: 0.9404
Epoch 56: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 4ms/step - loss: 0.1697 - accuracy: 0.9423 - val_loss: 2.7106e-04 - val_accuracy: 1.0000
Epoch 57/100
179/188 [===========================>..] - ETA: 0s - loss: 0.1466 - accuracy: 0.9487
Epoch 57: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 3ms/step - loss: 0.1453 - accuracy: 0.9489 - val_loss: 0.0019 - val_accuracy: 1.0000
Epoch 58/100
179/188 [===========================>..] - ETA: 0s - loss: 0.1532 - accuracy: 0.9483
Epoch 58: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 3ms/step - loss: 0.1510 - accuracy: 0.9484 - val_loss: 3.9394e-05 - val_accuracy: 1.0000
Epoch 59/100
178/188 [===========================>..] - ETA: 0s - loss: 0.1459 - accuracy: 0.9515
Epoch 59: val_loss did not improve from 0.00000
188/188 [===================

Epoch 85/100
181/188 [===========================>..] - ETA: 0s - loss: 0.1638 - accuracy: 0.9413
Epoch 85: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 3ms/step - loss: 0.1627 - accuracy: 0.9419 - val_loss: 4.0332e-06 - val_accuracy: 1.0000
Epoch 86/100
184/188 [============================>.] - ETA: 0s - loss: 0.1416 - accuracy: 0.9467
Epoch 86: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 3ms/step - loss: 0.1417 - accuracy: 0.9464 - val_loss: 2.1259e-06 - val_accuracy: 1.0000
Epoch 87/100
182/188 [============================>.] - ETA: 0s - loss: 0.1484 - accuracy: 0.9463
Epoch 87: val_loss did not improve from 0.00000
188/188 [==============================] - 1s 3ms/step - loss: 0.1511 - accuracy: 0.9448 - val_loss: 1.3430e-05 - val_accuracy: 1.0000
Epoch 88/100
180/188 [===========================>..] - ETA: 0s - loss: 0.1410 - accuracy: 0.9509
Epoch 88: val_loss did not improve from 0.00000
188/188 [==

In [95]:
metadata_evl=pd.read_csv("C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_eval_part1/labels_eval_part1.csv")#READ EVALUATION PART CSV

In [96]:
from numpy import loadtxt
from keras.models import load_model
model = load_model('C:/Users/admin/Desktop/saved_models/audio_classification_Part1.hdf5')#LOAD MODEL

In [97]:
audio_dataset_path_evl="C:/Users/admin/Desktop/spcup_2022_training_part1/spcup_2022_eval_part1"#SET PATH FOR EVALUATION PART

In [98]:
#FEATURE EXTRCTION FOR EVALUATION PART
predction_label=[]
for index,row in tqdm(metadata_evl.iterrows()):
    file_name=os.path.join(os.path.abspath(audio_dataset_path_evl),str(row["track"]))
    data=feature_extractor(file_name)
    data=data.reshape(1,-1)
    predict_x=model.predict(data) 
    classes_x=np.argmax(predict_x,axis=1)
    predction_label.append([str(row["track"]),int(classes_x)])

9000it [22:21,  6.71it/s]


In [99]:
predction_label=pd.DataFrame(predction_label,columns=['track','algorithm'])

In [102]:
#SCORES FILE CSV
from pathlib import Path  
filepath = Path('C:/Users/admin/Desktop/part1_scores.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
predction_label.to_csv(filepath) 

In [101]:
#TEXT FILE
#with open('C:/Users/admin/Desktop/answer.txt', 'w', encoding = 'utf-8') as f:
#    for rec_index, rec in predction_label.iterrows():
#        f.write(rec["track"]+', '+str(rec['algorithm']) + '\n')